<a href="https://colab.research.google.com/github/aSafarpoor/OSN_FAD/blob/main/ADA/Adversaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#initialization

In [2]:
SEED = 1

In [3]:
import numpy as np
import random
from sklearn.metrics import confusion_matrix, accuracy_score
import networkx as nx
from tqdm import tqdm

In [4]:
def load_txt_file(filename):
	try:
		with open(filename, 'r') as file:
			return [int(line.strip()) for line in file]
	except:
		with open(filename, 'r') as file:
			first_line = file.readline().strip()  # Read only the first line
			return [int(num) for num in first_line.split()]

def load_txt_file_for_edges(filename):
	c = 0
	with open(filename, 'r') as file:
		return [list(map(int,line.strip().split())) for line in file]

In [5]:
def partitioner(x,ratio,do_shuffle=True):
    midpoint = int(len(x) * ratio)

    # Split the list into two parts
    if do_shuffle:
        random.seed(SEED)
        random.shuffle(x)
    return x[:midpoint], x[midpoint:]


In [6]:
edges = load_txt_file_for_edges('edges.txt')

In [7]:
bknown = load_txt_file('btrain.txt')
sknown = load_txt_file('strain.txt')


btest = load_txt_file('btest.txt')
stest = load_txt_file('stest.txt')

nodes = list(set(np.array(edges).reshape(-1)))
num_nodes = len(nodes)

print(len(bknown),len(sknown),len(btest),len(stest))

20000 10000 158377 81263


# Data Generation

In [8]:
def remap_graph_and_update_lists(g,nodes_in_order, list_of_lists):
    # This function rename nodes to have n nodes with names {0,1,...,n-1}
    # and update edges based on that

    nodes = nodes_in_order
    node_mapping = {old_node: new_index for new_index, old_node in tqdm(enumerate(nodes))}

    g_mapped = nx.relabel_nodes(g, node_mapping)

    outlists = []
    for list_i in tqdm(list_of_lists):
        temp = [node_mapping[node] for node in list_i if node in node_mapping]
        outlists.append(temp[:])

    return g_mapped, outlists

We will have:
+ bknown_test(part of trainset moved for known in test),btrain,btest(labels)
+ sknown_test(part of trainset moved for known in test),strain,stest(labels)
+ unknowntrain,unknowntest
+ target (part of sybils, can be in test or train)
+ trainX,testX,trainY,testY
+ train_edges,test_edges

we seperated main btrain and strain to know nodes for train and test.

It is generally for inductive learning. For transductive you can add xknown_test back to ktrain


In [9]:
def sample_targets(stest,ratio):

    sybils = list(set(stest))

    random.seed(SEED)
    targets = random.sample(sybils[:], int(len(sybils)*ratio))

    return targets

In [10]:
def partitioner_transductive_inductive(bknown,sknow,btest,stest,edges,nodes,ratio,is_transductive):
    btrain,bknown_test = partitioner(bknown, ratio = ratio)
    strain,sknown_test = partitioner(sknown, ratio = ratio)

    knownset = set(bknown+sknown)
    unknownset = set(nodes)- knownset

    unknowntrain,unknowntest = partitioner(list(unknownset)[:], ratio = ratio)
    del(unknownset)

    trainX = btrain+strain+unknowntrain
    trainY = [0 for i in btrain]+[1 for i in strain]+[[-1 for i in unknowntrain]]
    testX = btest+stest+unknowntest+bknown_test+sknown_test+trainX
    textCompleteY = [0 for i in btest]+[1 for i in stest]+[-1 for i in unknowntest]+[0 for i in bknown_test]+[1 for i in sknown_test]+trainY
    textInitY = [-1 for i in btest]+[-1 for i in stest]+[-1 for i in unknowntest]+[0 for i in bknown_test]+[1 for i in sknown_test]+trainY

    if is_transductive:
        return (bknown_test,
                btrain,
                btest,
                sknown_test,
                strain,
                stest,
                unknowntrain,
                unknowntest,
                bknown_test,
                sknown_test,
                edges,
                trainX,
                trainY,
                testX,
                textCompleteY,
                textInitY
                )

    else:
        g = nx.Graph()
        g.add_edges_from(edges)


        gtrain = g.subgraph(trainX)
        gtest = g.subgraph(testX)

        gtrain, [btrain,strain,unknowntrain] = remap_graph_and_update_lists(g=gtrain,
                                                                            nodes_in_order = btrain+strain+unknowntrain,
                                                                            list_of_lists = [btrain,strain,unknowntrain])

        gtest, [btest,stest,unknowntest,bknown_test,sknown_test] = remap_graph_and_update_lists(g=gtest,
                                                                                                nodes_in_order=btest+stest+unknowntest+bknown_test+sknown_test,
                                                                                                list_of_lists=[btest,stest,unknowntest,bknown_test,sknown_test])

        train_edges = list(gtrain.edges())
        test_edges = list(gtest.edges())

        trainX = btrain+strain+unknowntrain
        trainY = [0 for i in btrain]+[1 for i in strain]+[-1 for i in unknowntrain]
        testX = btest+stest+unknowntest+bknown_test+sknown_test
        textCompleteY = [0 for i in btest]+[1 for i in stest]+[-1 for i in unknowntest]+[0 for i in bknown_test]+[1 for i in sknown_test]
        textInitY = [-1 for i in btest]+[-1 for i in stest]+[-1 for i in unknowntest]+[0 for i in bknown_test]+[1 for i in sknown_test]

        return (bknown_test,
                btrain,
                btest,
                sknown_test,
                strain,
                stest,
                unknowntrain,
                unknowntest,
                bknown_test,
                sknown_test,
                train_edges,
                test_edges,
                trainX,
                trainY,
                testX,
                textCompleteY,
                textInitY
                )


In [11]:
# # (
# #     bknown_test,
# #     btrain,
# #     btest,
# #     sknown_test,
# #     strain,
# #     stest,
# #     unknowntrain,
# #     unknowntest,
# #     bknown_test,
# #     sknown_test,
# #     edges,
# #     trainX,
# #     trainY,
# #     testX,
# #     textCompleteY,
# #     textInitY
# # ) = partitioner_transductive_inductive(bknown[:], sknown[:], btest[:], stest[:], edges[:], nodes[:], ratio=0.3, is_transductive=True)


# (
#     bknown_test,
#     btrain,
#     btest,
#     sknown_test,
#     strain,
#     stest,
#     unknowntrain,
#     unknowntest,
#     bknown_test,
#     sknown_test,
#     train_edges,
#     test_edges,
#     trainX,
#     trainY,
#     testX,
#     textCompleteY,
#     textInitY
# ) = partitioner_transductive_inductive(bknown[:], sknown[:], btest[:], stest[:], edges[:], nodes[:], ratio=0.3, is_transductive=False)

# Adversarial Attacks

In [12]:
def changing_edges_random(g,targets,addbudget,removebudget):
    random.seed(SEED)
    G = g.copy()

    allnodes = list(G.nodes())
    edge_dictionary = {}
    for t in targets:
        try:
            edge_dictionary[t] = [v for u, v in G.out_edges(t)]
        except:
            edge_dictionary[t] = []
    c = 0
    while c < addbudget:
        x,y = random.sample(targets, 2)
        if G.has_edge(x, y):
            pass
        else:
            G.add_edge(x,y)
            c+=1
    c = 0

    while c < removebudget:
        x = random.sample(targets,1)[0]

        if len(edge_dictionary[x])==0:
            continue

        y = random.sample(edge_dictionary[x],1)[0]
        try:
            G.remove_edge(x,y)
            edge_dictionary[x].remove(y)
            c+=1
        except:
            pass

    return G



In [13]:
def virtual_nodes_random(g,targets,edge_budget,node_budget,probability_of_virtual_to_virtual):
    random.seed(SEED)
    G = g.copy()

    allnodes = list(G.nodes())
    maxallnodes = max(allnodes)

    virtualnodes = [i+maxallnodes for i in range(1,node_budget+1)]
    G.add_nodes_from(virtualnodes)

    random.seed(SEED)

    c = 0
    while c < edge_budget:
        x = random.sample(virtualnodes,1)[0]
        if random.random() < probability_of_virtual_to_virtual:
            y = random.sample(virtualnodes,1)
            while(x==y):
                y = random.sample(virtualnodes,1)
        else:
            y = random.sample(targets,1)

        y = y[0]
        if G.has_edge(x,y):
            pass
        else:
            c+=1
            G.add_edge(x,y)

    return G,virtualnodes

In [14]:
(
    bknown_test,
    btrain,
    btest,
    sknown_test,
    strain,
    stest,
    unknowntrain,
    unknowntest,
    bknown_test,
    sknown_test,
    train_edges,
    test_edges,
    trainX,
    trainY,
    testX,
    textCompleteY,
    textInitY
) = partitioner_transductive_inductive(bknown[:], sknown[:], btest[:], stest[:], edges[:], nodes[:], ratio=0.3, is_transductive=False)



80892it [00:00, 1206833.82it/s]
100%|██████████| 3/3 [00:00<00:00, 52.12it/s]
428388it [00:00, 727699.10it/s]
100%|██████████| 5/5 [00:00<00:00, 22.81it/s]


In [15]:

g = nx.DiGraph()
g.add_edges_from(test_edges)

random.seed(SEED)
targets = sample_targets(stest,ratio=0.1)



random.seed(SEED)
G1 = changing_edges_random(g,targets,addbudget=100,removebudget=100)

random.seed(SEED)
G2,virtualnodes = virtual_nodes_random(g,targets,edge_budget=100,node_budget=10,probability_of_virtual_to_virtual=0.8)

random.seed(SEED)
G3,virtualnodes = virtual_nodes_random(G1,targets,edge_budget=100,node_budget=10,probability_of_virtual_to_virtual=0.8)

In [16]:
# g = nx.DiGraph()
# g.add_edges_from(test_edges)

# random.seed(SEED)
# targets = sample_targets(stest,ratio=0.1)
# G1 = changing_edges_random(g,targets,addbudget=100,removebudget=100)

# random.seed(SEED)
# G2,virtualnodes = virtual_nodes_random(G1,targets,edge_budget=100,node_budget=10,probability_of_virtual_to_virtual=0.8)

#classification experiance

In [17]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.4 MB/s eta 0:00:00


In [18]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric.utils as pyg_utils
from torch_geometric.nn import GATConv
from sklearn.metrics import accuracy_score

from torch_geometric.utils import subgraph
from torch_geometric.nn import SAGEConv

In [19]:
class Edited_GCN_ETH(torch.nn.Module):
    def __init__(self, num_node_features, num_layers, hidden_width, dropout=True):
        super(Edited_GCN_ETH, self).__init__()

        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        input_width = num_node_features
        self.num_classes = 2  # Assuming binary classification

        for i in range(num_layers):
            if i == 0:
                self.convs.append(GCNConv(input_width, hidden_width, bias=True))
            elif i == num_layers - 1:
                self.convs.append(GCNConv(hidden_width, self.num_classes, bias=True))
            else:
                self.convs.append(GCNConv(hidden_width, hidden_width, bias=True))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        h = x

        for i, conv in enumerate(self.convs):
            h = conv(h, edge_index)

            if i < len(self.convs) - 1:  # Apply activation and dropout for all but last layer
                h = F.relu(h)
                if self.dropout:
                    h = F.dropout(h, p=0.5, training=self.training)

        return F.log_softmax(h, dim=1)


class GAT_ETH(torch.nn.Module):
    def __init__(self, input_width, num_layers, hidden_width, num_classes, num_heads, dropout: bool = True):
        super().__init__()
        self.dropout = dropout
        self.num_classes = num_classes
        self.convs = torch.nn.ModuleList()

        for i in range(num_layers):
            if i == 0:
                self.convs.append(GATConv(input_width, hidden_width, heads=num_heads))
            elif i == num_layers - 1:
                self.convs.append(GATConv(hidden_width * num_heads, self.num_classes, heads=1))
            else:
                self.convs.append(GATConv(hidden_width * num_heads, hidden_width, heads=num_heads))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        h = x

        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
                # h = F.elu(h)
            i += 1

        return F.log_softmax(h, dim=1)

In [20]:
def main_inductive(model,
                   epochs,
                   train_nodes,
                   edge_index_train,
                   test_nodes,
                   edge_index_test,
                   btrain,
                   strain,
                   btest,
                   stest,
                   trainY,
                   testY,
                   bknown_test,
                   sknown_test,
                   targets = [],
                   virtualnodes = []):

    labels_train = torch.tensor(trainY, dtype=torch.long)
    labels_test = torch.tensor(testY, dtype=torch.long)

    edge_index_train = torch.tensor([[e[0] for e in edge_index_train[:]], [e[1] for e in edge_index_train[:]]], dtype=torch.long)
    edge_index_test = torch.tensor([[e[0] for e in edge_index_test[:]], [e[1] for e in edge_index_test[:]]], dtype=torch.long)



    # Initialize the node features
    x = torch.zeros((len(train_nodes), 1))  # 1-dimensional embeddings, all zeros initially
    for node in train_nodes:
        x[node] = 0.5  # Test nodes
    for node in btrain:
        x[node] = 0  # Benign training nodes
    for node in strain:
        x[node] = 1  # Sybil training nodes

    # Create the PyTorch Geometric data object for training
    data_train = Data(x=x, edge_index=edge_index_train, y=labels_train)


    # Create modified node features for test phase
    x = torch.zeros((len(test_nodes), 1))  # 1-dimensional embeddings, all zeros initially
    for node in test_nodes:
        x[node] = 0.5  # Test nodes
    for node in bknown_test:
        x[node] = 0  # Benign training nodes
    for node in sknown_test:
        x[node] = 1  # Sybil training nodes

    # Create the PyTorch Geometric data object for testing
    data_test = Data(x=x, edge_index=edge_index_test, y=labels_test)

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Masks for training and testing
    train_mask = torch.zeros(len(train_nodes), dtype=torch.bool)
    test_mask = torch.zeros(len(test_nodes), dtype=torch.bool)
    target_mask = torch.zeros(len(test_nodes), dtype=torch.bool)


    # Assign masks for training and testing nodes
    train_mask[btrain + strain] = True
    test_mask[btest + stest] = True
    target_mask[targets] = True


    # Training loop
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(data_train)
        loss = loss_fn(out[train_mask], data_train.y[train_mask])  # Only consider training nodes for loss
        loss.backward()
        optimizer.step()
        return loss.item()

    # Testing function
    def test():
        model.eval()
        out = model(data_test)
        pred = out.argmax(dim=1)  # Get predictions
        test_acc = accuracy_score(data_test.y[test_mask].cpu(), pred[test_mask].cpu())

        out = model(data_train)
        pred = out.argmax(dim=1)  # Get predictions
        train_acc = accuracy_score(data_train.y[train_mask].cpu(), pred[train_mask].cpu())

        out = model(data_test)
        pred = out.argmax(dim=1)  # Get predictions
        target_acc = accuracy_score(data_test.y[target_mask].cpu(), pred[target_mask].cpu())
        return test_acc, train_acc, target_acc

    # Training the model

    for epoch in tqdm(range(epochs)):
        loss = train()
        test_acc, train_acc, target_acc = test()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}, Train Accuracy: {train_acc:.4f}, Target Accuracy: {target_acc:.4f}")


    # Final evaluation
    testtest_acc, train_acc, target_acc = test()
    print(f'Final Results - Test Accuracy: {test_acc:.4f}, Train Accuracy: {train_acc:.4f}, Target Accuracy: {target_acc:.4f}')



In [21]:
# simple version
main_inductive(model = Edited_GCN_ETH(num_node_features=1, num_layers=2, hidden_width=4),
                epochs=1000,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX,
                edge_index_test = test_edges,
                btrain = btrain,
                strain = strain,
                btest = btest,
                stest = stest,
                targets = targets,
                trainY = trainY,
                testY = textCompleteY,
                bknown_test = bknown_test,
                sknown_test = sknown_test)



  0%|          | 1/1000 [00:09<2:38:51,  9.54s/it]

Epoch 0, Loss: 0.6941, Test Accuracy: 0.1987, Train Accuracy: 0.6673, Target Accuracy: 0.0001


  1%|          | 11/1000 [00:45<53:07,  3.22s/it]

Epoch 10, Loss: 0.6472, Test Accuracy: 0.1986, Train Accuracy: 0.7474, Target Accuracy: 0.0002


  2%|▏         | 21/1000 [01:18<56:40,  3.47s/it]

...

Epoch 980, Loss: 0.3697, Test Accuracy: 0.1944, Train Accuracy: 0.9019, Target Accuracy: 0.1419


 99%|█████████▉| 991/1000 [1:00:23<00:33,  3.74s/it]

Epoch 990, Loss: 0.3326, Test Accuracy: 0.1955, Train Accuracy: 0.8994, Target Accuracy: 0.1308


100%|██████████| 1000/1000 [1:00:56<00:00,  3.66s/it]


Final Results - Test Accuracy: 0.1947, Train Accuracy: 0.9013, Target Accuracy: 0.1407


In [22]:

# random add/remove
main_inductive(model = Edited_GCN_ETH(num_node_features=1, num_layers=2, hidden_width=4),
                epochs=1000,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX,
                edge_index_test = list(G1.edges),
                btrain = btrain,
                strain = strain,
                targets = targets,
                btest = btest,
                stest = stest,
                trainY = trainY,
                testY = textCompleteY,
                bknown_test = bknown_test,
                sknown_test = sknown_test)


  0%|          | 1/1000 [00:04<1:15:12,  4.52s/it]

Epoch 0, Loss: 0.7125, Test Accuracy: 0.1987, Train Accuracy: 0.6667, Target Accuracy: 0.0000


  1%|          | 11/1000 [00:41<58:39,  3.56s/it]  

Epoch 10, Loss: 0.6733, Test Accuracy: 0.1987, Train Accuracy: 0.6667, Target Accuracy: 0.0000


  2%|▏         | 21/1000 [01:19<1:00:58,  3.74s/it]

...

Epoch 980, Loss: 0.3976, Test Accuracy: 0.1914, Train Accuracy: 0.9042, Target Accuracy: 0.1600


 99%|█████████▉| 991/1000 [1:01:02<00:27,  3.02s/it]

Epoch 990, Loss: 0.3546, Test Accuracy: 0.1960, Train Accuracy: 0.9001, Target Accuracy: 0.1333


100%|██████████| 1000/1000 [1:01:32<00:00,  3.69s/it]


Final Results - Test Accuracy: 0.1907, Train Accuracy: 0.9043, Target Accuracy: 0.1643


In [ ]:

#virtual nodes
main_inductive(model = Edited_GCN_ETH(num_node_features=1, num_layers=2, hidden_width=4),
                epochs=1000,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX+virtualnodes,
                edge_index_test = list(G2.edges),
                btrain = btrain,
                strain = strain,
                btest = btest,
                stest = stest,
                targets = targets,
                virtualnodes = virtualnodes,
                trainY = trainY,
                testY = textCompleteY+[1 for i in virtualnodes],
                bknown_test = bknown_test,
                sknown_test = sknown_test)


  0%|          | 1/1000 [00:03<1:00:28,  3.63s/it]

Epoch 0, Loss: 0.7027, Test Accuracy: 0.1987, Train Accuracy: 0.6667, Target Accuracy: 0.0000


  1%|          | 11/1000 [00:34<52:44,  3.20s/it]

Epoch 10, Loss: 0.6444, Test Accuracy: 0.1987, Train Accuracy: 0.7478, Target Accuracy: 0.0001


  2%|▏         | 21/1000 [01:05<51:02,  3.13s/it]

...

 98%|█████████▊| 981/1000 [53:52<01:03,  3.36s/it]

Epoch 980, Loss: 0.3301, Test Accuracy: 0.1959, Train Accuracy: 0.9047, Target Accuracy: 0.1325


 99%|█████████▉| 991/1000 [54:23<00:28,  3.15s/it]

Epoch 990, Loss: 0.3119, Test Accuracy: 0.1973, Train Accuracy: 0.8923, Target Accuracy: 0.1231


100%|██████████| 1000/1000 [54:53<00:00,  3.29s/it]


In [24]:

#virtual nodes + add/remove
main_inductive(model = Edited_GCN_ETH(num_node_features=1, num_layers=2, hidden_width=4),
                epochs=1000,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX+virtualnodes,
                edge_index_test = list(G3.edges),
                btrain = btrain,
                strain = strain,
                btest = btest,
                stest = stest,
                targets = targets,
                virtualnodes = virtualnodes,
                trainY = trainY,
                testY = textCompleteY+[1 for i in virtualnodes],
                bknown_test = bknown_test,
                sknown_test = sknown_test)



  0%|          | 1/1000 [00:02<48:46,  2.93s/it]

Epoch 0, Loss: 0.6966, Test Accuracy: 0.1987, Train Accuracy: 0.6667, Target Accuracy: 0.0000


  1%|          | 11/1000 [00:35<53:14,  3.23s/it]

Epoch 10, Loss: 0.6650, Test Accuracy: 0.1987, Train Accuracy: 0.6667, Target Accuracy: 0.0000


  2%|▏         | 21/1000 [01:05<51:11,  3.14s/it]

...

Epoch 980, Loss: 0.3438, Test Accuracy: 0.1911, Train Accuracy: 0.9026, Target Accuracy: 0.1559


 99%|█████████▉| 991/1000 [52:46<00:28,  3.12s/it]

Epoch 990, Loss: 0.3445, Test Accuracy: 0.1890, Train Accuracy: 0.9032, Target Accuracy: 0.1664


100%|██████████| 1000/1000 [53:16<00:00,  3.20s/it]


Final Results - Test Accuracy: 0.1918, Train Accuracy: 0.9016, Target Accuracy: 0.1505


In [ ]:
# g = nx.DiGraph()
# g.add_edges_from(test_edges)

# random.seed(SEED)
# targets = sample_targets(stest,ratio=0.1)

# simple version
main_inductive(model = GAT_ETH(input_width=1, num_layers=4, hidden_width=4, num_classes=2, num_heads=4, dropout= True),
                epochs=10,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX,
                edge_index_test = test_edges,
                btrain = btrain,
                strain = strain,
                btest = btest,
                stest = stest,
                trainY = trainY,
                targets = targets,
                testY = textCompleteY,
                bknown_test = bknown_test,
                sknown_test = sknown_test)


# random add/remove
main_inductive(model = GAT_ETH(input_width=1, num_layers=4, hidden_width=16, num_classes=2, num_heads=8, dropout= True),
                epochs=10,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX,
                edge_index_test = list(G1.edges),
                btrain = btrain,
                strain = strain,
                btest = btest,
                stest = stest,
                trainY = trainY,
                targets = targets,
                testY = textCompleteY,
                bknown_test = bknown_test,
                sknown_test = sknown_test)


#virtual nodes
main_inductive(model = GAT_ETH(input_width=1, num_layers=4, hidden_width=16, num_classes=2, num_heads=8, dropout= True),
                epochs=10,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX+virtualnodes,
                edge_index_test = list(G2.edges),
                btrain = btrain,
                strain = strain,
                btest = btest,
                stest = stest,
                targets = targets,
                virtualnodes = virtualnodes,
                trainY = trainY,
                testY = textCompleteY+[1 for i in virtualnodes],
                bknown_test = bknown_test,
                sknown_test = sknown_test)



#virtual nodes + add/remove
main_inductive(model = GAT_ETH(input_width=1, num_layers=4, hidden_width=16, num_classes=2, num_heads=8, dropout= True),
                epochs=10,
                train_nodes = trainX,
                edge_index_train = train_edges,
                test_nodes = testX+virtualnodes,
                edge_index_test = list(G3.edges),
                btrain = btrain,
                strain = strain,
                btest = btest,
                stest = stest,
                targets = targets,
                virtualnodes = virtualnodes,
                trainY = trainY,
                testY = textCompleteY+[1 for i in virtualnodes],
                bknown_test = bknown_test,
                sknown_test = sknown_test)



 10%|█         | 1/10 [00:26<03:54, 26.05s/it]

Epoch 0, Loss: 0.6957, Test Accuracy: 0.2143, Train Accuracy: 0.7759, Target Accuracy: 0.0805


 20%|██        | 2/10 [00:49<03:16, 24.57s/it]